In [39]:
import numpy as np
import matplotlib.pyplot as plt
import gym
import pixiedust
from gym.envs.registration import register
from qiskit.aqua.components.optimizers import ADAM,COBYLA
from qiskit.aqua.components.variational_forms.ry import RY
from qiskit import *
from datetime import datetime

In [2]:
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [3]:
register(
    id='FrozenLakeNotSlippery-v0',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name' : '4x4', 'is_slippery': False},
    max_episode_steps=100,
    reward_threshold=0.8196, # optimum = .8196, changing this seems have no influence
)

In [4]:
env = gym.make("FrozenLakeNotSlippery-v0")

In [5]:
def get_numStates(env):
    if type(env.observation_space) != gym.spaces.tuple.Tuple:
        return env.observation_space.n
    dim_list = []
    for sp in env.observation_space:
        dim_list.append(sp.n)
    dim_list = np.array(dim_list)
    return dim_list.prod()

In [6]:
def get_Statehash(env,state):
    if type(env.observation_space) != gym.spaces.tuple.Tuple:
        return state
    dim_list = []
    for sp in env.observation_space:
        dim_list.append(sp.n)
    dim_list = np.array(dim_list)
    h = 0
    for i in range(len(dim_list)-1):
        h += state[i]*dim_list[i+1:].prod()
    h += state[-1]
    return h
    

In [7]:
def QLearning(iterations,alpha, gamma, epsilon ,env):
    returnlist = []
    num_actions = env.action_space.n
    num_states = get_numStates(env)
    Q = np.zeros((num_states,num_actions))
    for it in range(iterations):
        state = env.reset()
        R = 0

        done = False
        while not done:
            state_h = get_Statehash(env,state)
            if np.random.random() < epsilon:
                action = env.action_space.sample()
            else:
                candidates = np.where(Q[state_h] == np.max(Q[state_h]))[0]
                action = np.random.choice(candidates)
            statep, reward, done, info = env.step(action)
            if reward == 0:
                if done:
                    reward = -0.2
                else:
                    reward = -0.01
            else:
                reward = 1.0
            R *= gamma
            R += reward
            statep_h = get_Statehash(env,statep)
            Q[state_h,action] += alpha*(reward + gamma*Q[statep_h].max() -Q[state_h,action])
            state = statep
            
        returnlist.append(R)
        if it%10000 == 0:
            print('Iteration %d, Reward: %d'%(it,R))
            
    return returnlist, Q
        
        
        



In [ ]:
rl, Q = QLearning(100000,1e-3,0.99,0.25,env)  

In [ ]:
#     backend_sim = Aer.get_backend('qasm_simulator')

#     result = execute(qc, backend_sim, shots = shots).result()

#     counts = result.get_counts()
#     expect = np.zeros(nqbits)
#     for c in counts :
#         for n in range(nqbits):
#             expect[n] += int(c[n])*counts[c]/shots
#     #print(counts)
#     return np.array(expect)

In [131]:
def get_Qvalues(s_list,theta):
#     shots = 1000
#     nqbits = 4
#     num_rep = 1
#     qc_list = []
#     for s in s_list:
#         q = QuantumRegister(nqbits)
#         c = ClassicalRegister(nqbits)
#         qc = QuantumCircuit(q,c)

#         d = np.binary_repr(int(s),nqbits)
#         for j,i in enumerate(d):
#             if i == '1':
#                 qc.x(q[j])

#         theta = theta.reshape((nqbits,3))

#         for rep in range(num_rep):
#             for i in range(1, nqbits):
#                 qc.cx(q[i-1], q[i])

#             for i in range(nqbits):
#                 qc.u3( theta[i][0], theta[i][1], theta[i][2],q[i])



#         #qc.measure(q,c)
        
#         qc_list.append(qc)
    shots = 1000
    depth = 4
    qc_list = []
    nqbits = 4
    theta.flatten()
    for s in s_list:
        q = QuantumRegister(nqbits)
        c = ClassicalRegister(nqbits)
        qc = QuantumCircuit(q,c)

        d = np.binary_repr(int(s),nqbits)
        for j,i in enumerate(d):
            if i == '1':
                qc.x(q[j])
        
        #qc += RY(nqbits,1, [[0,1],[1,2],[2,3]], 'linear', None, 'cx').construct_circuit(theta)        
        qc += RY(nqbits,depth).construct_circuit(theta,q)        
        
        #print(qc)
        
        qc_list.append(qc)


    
    backend_sim = Aer.get_backend('statevector_simulator')
    qobj = assemble(qc_list,backend_sim)
    #result_list = execute(qc_list, backend_sim, optimization_level = 0).result()
    job = backend_sim.run(qobj)
    result_list = job.result()
    expect_list = []
    for result in result_list.results:
        proba = abs(np.array(result.data.statevector))**2


        expect = np.zeros(nqbits)

        for c in range(len(proba)):
            cbin = np.binary_repr(int(c),nqbits)

            for n in range(nqbits) : 
                if cbin[n] == '1':
                    expect[n] += proba[c]
                    
        expect_list.append(np.flip(expect))
                
    #print(counts)
    return expect_list

In [ ]:
q = np.array([0.9,0.92,0.94,0.96,0.98,1.0] + [0]*6*3)
q/2 + 0.5

In [117]:
def loss(theta):
    su = 0
    
    s = [0] + [1] + [2] + [6] + [10] + [14]
    a = np.array([[0, 0, 1, 0], [0, 0, 1, 0],  [0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]])
    
    Qs = get_Qvalues(s,theta)
    #q = np.array([0.9,0.92,0.94,0.96,0.98,1.0] + [0]*6*3)
    #q = q/2 + 0.5
    #a = [2,2,1,1,1,2] + [0]*6 + [3]*6 + [1,1,2,2,2,1]
    #for i in range(len(q)):
    #    su += (q[i] - Qs[i][int(a[i])])**2
    #print(su/len(q))
    su = (np.abs(Qs - a)).mean()
    print(su)
    return su

In [ ]:
def loss_original(theta):
    su = 0
    Qs = get_Qvalues([0,1,2,6,10,14]+ [0,1,2,6,10,14]*3,theta)
    q = np.array([0.9,0.92,0.94,0.96,0.98,1.0] + [0]*6*3)
    q = q/2 + 0.5
    a = [2,2,1,1,1,2] + [0]*6 + [3]*6 + [1,1,2,2,2,1]
    for i in range(len(q)):
        su += (q[i] - Qs[i][int(a[i])])**2
    print(su/len(q))
    return su/len(q)

In [132]:
def DQN(iterations,alpha, gamma, epsilon ,env,N,batchsize):
    nqbits = 4
    depth = 4
    theta = np.array([2*np.pi*np.random.random(nqbits*(depth+1))]).flatten()
    
    #adam = ADAM(maxiter = iterations, lr = alpha)
    cobyla = COBYLA(maxiter = iterations)
    #theta,obj,_ = adam.optimize(2*nqbits,loss, initial_point = theta)
    theta,obj,_ = cobyla.optimize(nqbits*(depth+1),loss, initial_point = theta)
    s = env.reset()
    done = False
    total_reward = 0
    while not done:
        env.render()
        print(s)
        print(get_Qvalues([s],theta)[0])
        a = np.argmax(get_Qvalues([s],theta)[0])
        s1,r,done,_ = env.step(a)
        if r == 0:
            if done:
                r = -0.2
            else:
                r = -0.01
        else:
            r = 1.0
        #print(r)
        total_reward += r
        s = s1
    env.render()
    
        
    return obj, theta

In [133]:
total_rewards, theta = DQN(400,1e-1,0.99,0.9,env,100,10)

0.5057327624974001
0.4846988574193787
0.4802209187756634
0.5111933050689089
0.47589843721606306
0.4927097201728812
0.47007923905524046
0.5279355628371724
0.49787213182006046
0.464502390117824
0.4915395542114574
0.4757597348161143
0.4691131231338712
0.47506693782946524
0.4843384016416108
0.4657649574322455
0.46504837647017533
0.4622067069693727
0.5239714597219643
0.45907216282251156
0.4551648307379938
0.4198693114556648
0.44792991281945077
0.42648408420903877
0.4234034177982347
0.4383676878672051
0.44617051557588505
0.4390616709707707
0.41710364326663657
0.42196609710738375
0.43793422670118537
0.41773969996513044
0.4132463094618907
0.4119267236808042
0.4491298321714972
0.41208851784386075
0.42654316347393056
0.4089427913393017
0.4154446342771223
0.40632210851177075
0.39932252463299944
0.4004121127683246
0.42542320210356516
0.4190478543316752
0.3804128304291168
0.3695504666265251
0.36844408458262895
0.4225178208377576
0.35723536830821284
0.3613936093985333
0.35326286501786713
0.376824650

In [82]:
print(total_rewards)

0.29564234505690196


In [128]:
np.argmax(get_Qvalues([0,1,2,6,10,14],theta),axis = 1)

array([2, 2, 1, 1, 3, 3], dtype=int64)

In [ ]:
plt.plot(total_rewards,'x')
plt.show()

In [36]:
env.reset()

0

0 - left

1 - down

2 - right

3 - up

0|1|2|3
4|5|6|7
8|9|10|11
12|13|14|15

In [41]:
env.render()

  (Left)
SFFF
FHFH
FFFH
HFFG


In [42]:
env.step(1)


(4, 0.0, False, {'prob': 1.0})

In [44]:
-0.01-0.01*0.99 - 0.99*0.99*0.01 + 0.99*0.99*0.99

0.9405979999999999